In [1]:
import pandas as pd
from stage import fullPrint, getStage
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import pickle
import swifter
import os

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

PEAK = 13
allList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
allList = (nasdaqList + nyseList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [2]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        df = df[~df.index.duplicated()]
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nysenasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nysenasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
except:
    print("Something went wrong")

1/4328
2/4328
3/4328
4/4328
5/4328
6/4328
7/4328
8/4328
9/4328
10/4328
11/4328
12/4328
13/4328
14/4328
15/4328
16/4328
17/4328
18/4328
19/4328
20/4328
21/4328
22/4328
23/4328
24/4328
25/4328
26/4328
27/4328
28/4328
29/4328
30/4328
31/4328
32/4328
33/4328
34/4328
35/4328
36/4328
37/4328
38/4328
39/4328
40/4328
41/4328
42/4328
43/4328
44/4328
45/4328
46/4328
47/4328
48/4328
49/4328
50/4328
51/4328
52/4328
53/4328
54/4328
55/4328
56/4328
57/4328
58/4328
59/4328
60/4328
61/4328
62/4328
63/4328
64/4328
65/4328
66/4328
67/4328
68/4328
69/4328
70/4328
71/4328
72/4328
73/4328
74/4328
75/4328
76/4328
77/4328
78/4328
79/4328
80/4328
81/4328
82/4328
83/4328
84/4328
85/4328
86/4328
87/4328
88/4328
89/4328
90/4328
91/4328
92/4328
93/4328
94/4328
95/4328
96/4328
97/4328
98/4328
99/4328
100/4328
101/4328
102/4328
103/4328
104/4328
105/4328
106/4328
107/4328
108/4328
109/4328
110/4328
111/4328
112/4328
113/4328
114/4328
115/4328
116/4328
117/4328
118/4328
119/4328
120/4328
121/4328
122/4328
123/4328
1

In [3]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def calculateSpyDfSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,8]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [4]:
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf = spDf[~spDf.index.duplicated()]
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSpyDfSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply: 100%|██████████| 1372/1372 [00:00<00:00, 27810.41it/s]


In [5]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 712/712 [00:00<00:00, 27827.84it/s]


AACG


Pandas Apply: 100%|██████████| 19/19 [00:00<00:00, 20555.01it/s]


AACI


Pandas Apply: 100%|██████████| 31/31 [00:00<00:00, 26230.27it/s]


...

SCOA


Pandas Apply: 100%|██████████| 398/398 [00:00<00:00, 25157.99it/s]


SCYX


Pandas Apply: 100%|██████████| 1/1 [00:00<00:00, 1490.51it/s]

TOIIW


In [6]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [7]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [8]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

,open,high,low,close,adjclose,volume,ticker
2017-04-28,13.50,14.25,13.00,13.00,13.00,953900,ZYME
2017-05-05,13.16,13.20,13.00,13.05,13.05,20500,ZYME
2017-05-12,13.09,13.12,13.00,13.00,13.00,5400,ZYME
2017-05-19,13.03,13.05,13.00,13.04,13.04,7400,ZYME
2017-05-26,13.04,13.04,12.50,12.59,12.59,16500,ZYME
...,...,...,...,...,...,...,...
2022-02-18,7.06,7.38,7.04,7.18,7.18,633800,ZYME
2022-02-25,7.00,7.48,6.93,7.28,7.28,687600,ZYME
2022-03-04,6.35,6.54,6.10,6.18,6.18,637900,ZYME
2022-03-11,7.02,7.02,6.67,6.76,6.76,574800,ZYME


In [9]:
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(stockDataTickerList, f)

In [10]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [11]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
removeList = []
for ticker in nasdaqList:
    try:
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

['AACG', 'AACI', 'AADI', 'AAL', 'AAME', 'AAON', 'AAWW', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABGI', 'ABIO', 'ABMD', 'ABOS', 'ABSI', 'ABTX', 'ABUS', 'ABVC', 'ACABU', 'ACAD', 'ACAH', 'ACAXU', 'ACB', 'ACBA', 'ACBAU', 'ACBI', 'ACCD', 'ACER', 'ACET', 'ACEV', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHV', 'ACIU', 'ACLX', 'ACNB', 'ACOR', 'ACQR', 'ACQRU', 'ACRS', 'ACRX', 'ACST', 'ACT', 'ACTD', 'ACTG', 'ACXP', 'ADAG', 'ADALU', 'ADALW', 'ADAP', 'ADER', 'ADES', 'ADGI', 'ADIL', 'ADMA', 'ADMP', 'ADN', 'ADOC', 'ADP', 'ADPT', 'ADSE', 'ADTX', 'ADUS', 'ADVM', 'ADXN', 'AEAC', 'AEHL', 'AEIS', 'AEMD', 'AEP', 'AEPPZ', 'AERC', 'AERI', 'AEZS', 'AFAQU', 'AFBI', 'AFIB', 'AFMD', 'AFYA', 'AGEN', 'AGFS', 'AGFY', 'AGGR', 'AGIO', 'AGLE', 'AGRI', 'AGRX', 'AGTC', 'AHCO', 'AHG', 'AHPA', 'AHPAU', 'AHPI', 'AIH', 'AIHS', 'AIKI', 'AIMC', 'AINV', 'AIRS', 'AIRT', 'AIRTP', 'AKBA', 'AKIC', 'AKRO', 'AKTX', 'AKU', 'AKUS', 'AKYA', 'ALAC', 'ALBO', 'ALCO', 'ALDX', 'ALEC', 'ALGN', 'ALGS', 'ALGT', 'ALHC', 'ALIM', 'ALJJ', 'ALKS', 'ALL

In [3]:
for ticker in removeList:
    nasdaqList.append(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)

In [ ]:
import pandas as pd
df = pd.read_pickle("stockData/nasdaq.pkl")
print(df['USEG'])